# Mqtt python integration

In [ ]:
import paho.mqtt.client as mqtt
import pandas as pd

BROKER = "192.168.188.159"   
PORT = 1883
TOPIC = "SPPS-04/status/switch:0"  

messages = []

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        client.subscribe(TOPIC)
    else:
        print(f"Failed to connect, return code {rc}")

def on_message(client, userdata, msg):
    payload = msg.payload.decode()
    print(f"{msg.topic} => {payload}")
    messages.append({"topic": msg.topic, "message": payload})

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(BROKER, PORT, 60)

try:
    client.loop_start()
    print("Listening to MQTT...")
    while True:
        pass
except KeyboardInterrupt:
    print("\n Stopping...")
    client.loop_stop()

df = pd.DataFrame(messages)
print(df.head())


# Influxdb to python

In [ ]:
from influxdb_client import InfluxDBClient
import pandas as pd

# --- Connection details ---
url = "http://localhost:8086"
token = "aFqHDK6EKcy6HSgIBYU**********j_68KCm4HvKk5oyBByzORR55DeHwOyKwgww=="   
org = "microfactory"                
bucket = "sensor_data"          

# --- Connect to InfluxDB ---
client = InfluxDBClient(url=url, token=token, org=org)

# --- Define Flux query ---
query = f'''
from(bucket: "{bucket}")
  |> range(start: -1h)  // last 1 hour
  |> filter(fn: (r) => r["_measurement"] == "temperature")
'''

query_api = client.query_api()
tables = query_api.query_data_frame(org=org, query=query)

if isinstance(tables, list):
    df = pd.concat(tables)
else:
    df = tables

print(df.head())
